In [1]:
!pip install transformers

from pathlib import Path
import os

     |████████████████████████████████| 2.5MB 15.0MB/s 
     |████████████████████████████████| 3.3MB 45.7MB/s 
     |████████████████████████████████| 901kB 46.7MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pwd

/content


In [4]:
from google.colab import auth
auth.authenticate_user()

# Shared

In [5]:
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
digits = "([0-9])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|me|edu)"


def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [6]:
yung_test = 'This is just a random string of words to test the sentence splitter. Here we go. "Hey mom dont forget to drop off the boys." "Yeah dont worry about it dude." True.'

In [7]:
split_into_sentences(yung_test)

['This is just a random string of words to test the sentence splitter.',
 'Here we go.',
 '"Hey mom dont forget to drop off the boys".',
 '"Yeah dont worry about it dude".',
 'True.']

# NER Preprocessing of goodreads maharjan

In [ ]:
dataset_base = Path(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed")
data_yaml = dataset_base / "train_test_val_split_goodreads.yaml"

def get_next_book():
    with open(data_yaml) as file:
        documents = yaml.full_load(file)

        for train_test, doc in documents.items():
            for d in doc:
                for path in dataset_base.rglob(d):
                    with open(path, 'r', encoding = 'utf-8') as file:
                        yield file.read(), path

In [ ]:
book_gen = get_next_book()

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

nlp = pipeline("ner", grouped_entities=True, device = 0)

In [ ]:
import csv
import pandas as pd

In [ ]:
counter = 0
base_input_dir = Path(r"/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed")
base_output_dir = Path(r'/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed_and_nered')
for book, path in book_gen:
  print(path)

  file_name = os.path.basename(path)
  save_to = os.path.join(base_output_dir, path.relative_to(base_input_dir))
  if save_to.endswith('.txt'):
    save_to = save_to[:-4]
    save_to = save_to + '.csv'

  split_book = split_into_sentences(book)
  for i in range(len(split_book)):
    sent = split_book[i]
    ner_results = nlp(sent)
    for ner_r in reversed(ner_results):
      if ner_r['entity_group'] == 'PER':
        split_book[i] = sent[:ner_r['start']] + "[CHARACTER]" + sent[ner_r['end']:]
  
  dummy_df = pd.DataFrame(split_book)
  dummy_df.to_csv(save_to, index = False)

  if counter % 50 == 0:
    print(counter)
  
  counter = counter + 1

/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed/Short_stories/success/13724_the+frontiersmen.txt
0
/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed/Drama/failure/4765_loyalties.txt
/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed/Short_stories/failure/23588_a+filbert+is+a+nut.txt
/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed/Love_stories/success/18057_flower+of+the+dusk.txt
/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed/Poetry/success/109_renascence+and+other+poems.txt
/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed/Drama/success/14591_faust.txt
/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed/Poetry/success/14591_faust.txt
/content/drive/MyDrive/Thesis/Datasets/book_preprocessing/goodreads_maharjan_trimmed/Poetry/failure/27179_lam

# NER Preprocessing of Goodreads Guarro

In [8]:
import sqlite3
conn = sqlite3.connect('/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/guarro_goodreads.db')
dataCur = conn.cursor()
dataCur.execute("Select file_path from books where use_for_adaptive_ft = 2")

In [9]:
import os, os.path
import errno
import pathlib

# Taken from https://stackoverflow.com/a/600612/119527
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else: raise

def safe_open_w(path):
    ''' Open "path" for writing, creating any parent directories as needed.
    '''
    mkdir_p(os.path.dirname(path))
    return open(path, 'w', encoding='utf-8')

In [10]:
# basepath = Path(r'C:\Users\lucag\Google Drive\Thesis\Datasets\project_gutenberg')
basepath = Path('/content/drive/MyDrive/Thesis/Datasets/project_gutenberg')

def get_next_book():
    record = dataCur.fetchone()
    while (record):
        file_path = basepath / pathlib.PureWindowsPath(record[0]).as_posix()
        trimmed_path = file_path.parent / 'preprocessed' / 'trimmed' / file_path.name
        print (file_path)
        data = None
        try:
            with open(trimmed_path, encoding='utf-8') as myfile:
                data=myfile.read()
        except UnicodeDecodeError:
            with open(trimmed_path, encoding='latin-1') as myfile:
                data=myfile.read()
        
        record = dataCur.fetchone()
        yield data, file_path

In [11]:
book_gen = get_next_book()

In [12]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

nlp = pipeline("ner", grouped_entities=True, device = 0)

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:155: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


In [ ]:
counter = 0

for book, path in book_gen:
  save_to = path.parent / 'preprocessed' / 'nered' / path.name
  if not os.path.exists(save_to):
    print(save_to)
    split_book = split_into_sentences(book)
    for i in range(len(split_book)):
      sent = split_book[i]
      ner_results = nlp(sent)
      for ner_r in reversed(ner_results):
        if ner_r['entity_group'] == 'PER':
          split_book[i] = sent[:ner_r['start']] + "[CHARACTER]" + sent[ner_r['end']:]
    
    NERed_book = ' '.join(split_book)

    with safe_open_w(save_to) as f:
        f.write(NERed_book)
    # text_file = open(str(save_to), 'w', encoding = "utf-8")
    # text_file.write(NERed_book)
    # text_file.close()

    if counter % 100 == 0:
      print(counter)

    counter = counter + 1
  else:
    pass
    # print("skipping " + str(path))

/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/1/13/13-0.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/2/24/24-0.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/3/35/35-0.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/4/45/45-0.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/4/47/47-0.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/5/54/54.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/5/58/58.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/6/62/62.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/6/64/64.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/7/73/73.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/7/77/77-0.txt
/content/drive/MyDrive/Thesis/Datasets/project_gutenberg/All65kBooks/8/82/82-0.txt
/content/drive

# Preprocessing of bookcorpus

In [ ]:
dataset_base = Path(r"/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/epubtxt")
base_output_dir = Path(r'/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed')

def get_next_book():
  for file in os.listdir(dataset_base):
    output_file = base_output_dir / file
    if file.endswith(".txt"):
        if not os.path.exists(output_file):
          bookpath = os.path.join(dataset_base, file)
          with open(bookpath, 'r', encoding = 'utf-8') as book:
              yield book.read(), bookpath


## Removing non-english books

In [ ]:
book_gen = get_next_book()

In [ ]:
!pip install langdetect

     |████████████████████████████████| 983kB 13.6MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993223 sha256=89e6902bd0da76d3101885112f669c2195bf4c0b727de4af3bc2be8c1e6d5b85
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
Successfully built langdetect


In [ ]:
from langdetect import detect

In [ ]:
non_eng_books = []
exception_books = []

In [ ]:
from IPython.display import clear_output 

for book, path in book_gen:
  split_book = split_into_sentences(book)
  third_way = len(split_book)//3
  sample_to_test = split_book[third_way:third_way + 5]
  sample_to_test = ' '.join(sample_to_test)
  try:
    book_lang = detect(sample_to_test)
    if book_lang != 'en':
      print('non-eng book detected:', path, ' in ', book_lang)
      non_eng_books.append(path)
  except Exception as e:
    exception_books.append(path)

In [ ]:
len(non_eng_books)

108

In [ ]:
exception_books

['/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/epubtxt/in-sickness-and-in-health.epub.txt',
 '/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/epubtxt/secrets-of-abuse-survival.epub.txt',
 '/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/epubtxt/the-son-can-dream.epub.txt',
 '/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/epubtxt/four-windows-seattle-part-1.epub.txt',
 '/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/epubtxt/the-imbecile.epub.txt',
 '/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/epubtxt/jesus-christ.epub.txt']

In [ ]:
for f in non_eng_books:
  os.remove(f)

## NERing all the books

In [ ]:
book_gen = get_next_book()

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

nlp = pipeline("ner", grouped_entities=True, device = 0)

In [ ]:
counter = 0

for book, path in book_gen:

  file_name = os.path.basename(path)
  save_to = base_output_dir / file_name

  if not os.path.exists(save_to):
    print(save_to)
    split_book = split_into_sentences(book)
    for i in range(len(split_book)):
      sent = split_book[i]
      ner_results = nlp(sent)
      for ner_r in reversed(ner_results):
        if ner_r['entity_group'] == 'PER':
          split_book[i] = sent[:ner_r['start']] + "[CHARACTER]" + sent[ner_r['end']:]
    
    NERed_book = ' '.join(split_book)
    text_file = open(str(save_to), 'w', encoding = "utf-8")
    text_file.write(NERed_book)
    text_file.close()

    if counter % 100 == 0:
      print(counter)

    counter = counter + 1
  else:
    print("skipping " + file_name)

/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/a-lonely-man.epub.txt
0
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/how-to-find-a-piano-teacher.epub.txt
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/a-slice-of-the-big-apple.epub.txt
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/outlawed-songs-and-poems.epub.txt
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/picking-up-the-pieces.epub.txt
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/dead-end-of-a-circle.epub.txt
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/song-walker-starseeds-one.epub.txt
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/awareness-and-the-story-behind-addiction.epub.txt
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/rylaes-storm.epub.txt
/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/the

In [ ]:
open(fname, 'w', encoding='utf-8')

In [ ]:
import io

In [ ]:
f = io.open("/content/drive/MyDrive/Thesis/Datasets/bookcorpus/books1/preprocessed/laments-and-liaisons.epub.txt", "w")
f.write("Now the file has more content!")
f.close()

In [ ]:
open

<function io.open>

# Preprocessing of Project Gutenberg